# **PreProcess**
테이블 자료간 연결을 위한 전처리 및 Index 작업

In [9]:
from glob import glob
file_list = glob("data/food*.csv")
", ".join(file_list)

'data/food_recall.csv, data/food_nutrient.csv, data/food_recipe.csv, data/food_barcode.csv'

# **레시피 데이터**
## **1 레시피 데이터 전처리**
데이터 프레임 자료 불러오기

In [2]:
# 데이터 불러오기
import json
with open("data/food_recipie.json", "r", encoding='utf-8-sig') as f:
    recipe = f.read()
food_recipe = json.loads(recipe)

In [3]:
# Json 데이터를 DataFrame 변환
import pandas as pd
food_menuset = [(m,k,v) for m, _ in food_recipe.items()  
                        for k,v in _.items()]
food_menuset = pd.DataFrame(food_menuset)
food_menuset.columns = ['메뉴', '식재료명', '용량(1인/g)']
food_menuset.insert(0, '소유자', '뭐먹지연구소')
food_menuset.insert(2, '식재료구분', '재료/양념')
food_menuset.insert(5, '원재료', '')
food_menuset.insert(6, '알러지', '')
food_menuset.to_csv('data/lime_recipe.csv', index=None)
print(food_menuset.shape); food_menuset.head(15)

(1886, 7)


,소유자,메뉴,식재료구분,식재료명,용량(1인/g),원재료,알러지
0,뭐먹지연구소,멕시칸샐러드,재료/양념,스모크햄,10,,
1,뭐먹지연구소,멕시칸샐러드,재료/양념,양배추,20,,
2,뭐먹지연구소,멕시칸샐러드,재료/양념,당근,10,,


In [5]:
# 레시피 재료가 원재료와 다른경우 처리가 필요
food_items = sorted(set(food_menuset['식재료명']))
len(food_items), ", ".join(food_items)[::10]

(497,
 ' 가,자산고,소건,,래콩,  ,,,약,김,,김,루/가가이김루깐  깻게  ,,  닭갈닭근당대 ,근,끼닭리까목민루/돈    돈 사골 ,  ,,,분땅 ,페 마 부김민치스고  ,, 김 묵,빵역민민/민루이락 이배 아봄대 부당,,시블채어육,이추이약   박물,크위스스면시식치  감알 ,호 야   ,추이 양피 얼유시렌오액오이,클어스수온와요 명원반기,,스,,,알목단,,제 ,순 쥐진파상빵름/나캔쌀 소, 추  ,취치치,,레국케콩말반링위타  디 ,분인원 원파 트,팽,,상,,고표  지피이 ,해그해/씨 라박라합홍홍태후후 ,')

## **2 식재료 정보1**
재료별 영양소 정보

In [5]:
import pandas as pd
food_nutr = pd.read_csv("data/food_nutrient.csv")
col_filter = ['식품이름', '열량(kcal)(1회제공량당)',
       '탄수화물(g)(1회제공량당)', '단백질(g)(1회제공량당)', '지방(g)(1회제공량당)', '당류(g)(1회제공량당)',
       '나트륨(mg)(1회제공량당)', '콜레스테롤(mg)(1회제공량당)', '포화지방산(g)(1회제공량당)',
       '트랜스지방(g)(1회제공량당)']
food_nutr = food_nutr.loc[:, col_filter]
food_nutr.columns = [_.replace("1회제공량","100g")  for _ in food_nutr.columns]
food_nutr.head(3)

,식품이름,열량(kcal)(100g당),탄수화물(g)(100g당),단백질(g)(100g당),지방(g)(100g당),당류(g)(100g당),나트륨(mg)(100g당),콜레스테롤(mg)(100g당),포화지방산(g)(100g당),트랜스지방(g)(100g당)
0,"고량미,알곡",349.0,67.8,10.1,3.7,NaN,NaN,NaN,NaN,NaN
1,"겉귀리,생것",332.0,73.5,11.4,3.7,NaN,2,NaN,NaN,NaN
2,"쌀귀리,생것",334.0,70.4,14.3,3.8,NaN,3,NaN,NaN,NaN


In [13]:
import re
temp = "트랜스지방(g)(1회제공량당)"

In [16]:
re.findall("1회제공량", temp)

['1회제공량']

In [11]:
for _ in food_nutr.columns[1:]


Index(['식품이름', '열량(kcal)(1회제공량당)', '탄수화물(g)(1회제공량당)', '단백질(g)(1회제공량당)',
       '지방(g)(1회제공량당)', '당류(g)(1회제공량당)', '나트륨(mg)(1회제공량당)',
       '콜레스테롤(mg)(1회제공량당)', '포화지방산(g)(1회제공량당)', '트랜스지방(g)(1회제공량당)'],
      dtype='object')

In [7]:
food_nutr.columns

Index(['번호', '식품코드', '식품군', '식품이름', '1회제공량(g)', '열량(kcal)(1회제공량당)',
       '탄수화물(g)(1회제공량당)', '단백질(g)(1회제공량당)', '지방(g)(1회제공량당)', '당류(g)(1회제공량당)',
       '나트륨(mg)(1회제공량당)', '콜레스테롤(mg)(1회제공량당)', '포화지방산(g)(1회제공량당)',
       '트랜스지방(g)(1회제공량당)', '가공업체명', '구축년도', '자료원'],
      dtype='object')

In [8]:
",".join(sorted(set(food_nutr["식품군"].to_list())))

'가공유류,가공치즈,감자 및 전분류,견과류,곡류 및 그 제품,곡류, 서류 제품,과·채가공품류,과실류,과일·채소류음료,과일류,과자,과자류,구이류,국 및 탕류,규격외일반가공식품,기타,기타음료,김치류,나물/숙채류,난류,다류,당류,당류 및 그 제품,두류 및 그 제품,두류, 견과 및 종실류,두부류 또는 묵류,두유류,드레싱류,땅콩 또는 견과류 가공품류,만두류,면 및 만두류,면류,발효유류,발효음료류,밥류,버섯류,버터류,복합조미식품,볶음류,분식,분유류,빵 또는 떡류,빵 및 과자류,생채/무침류,성장기용 조제식,소스류,소시지류,수조어육류,시리얼류,식물성크림,식용유지류,식육 또는 알가공품,식육 추출가공품,아이스크림,아이스크림류,아이스크림믹스류,어육가공품,어패류,엿류,영·유아용 곡류조제식,올리고당류,우유 및 유제품,우유가공품,우유류,유가공품,유당분해우유,유제품류 및 빙과류,유지류,유크림류,육류 및 그 제품,음료,음료 및 주류,음료 및 차류,인삼·홍삼음료,자연치즈,장기보존식품,장아찌/절임류,잼류,저지방우유류,전/ 적 및 부침류,젓갈류,제빵,조리가공식품류,조림류,조미김,조미료류,조제유류,죽 및 스프류,즉석섭취·편의식품류,찌개 및 전골류,찜류,채소, 해조류,채소류,체중조절용 조제식품,추출가공식품류,카레,커피,코코아가공품류 또는 초콜릿류,탄산음료류,튀김류,특수의료용도등식품,팝콘용옥수수가공품,피자,해조류,햄버거'

In [9]:
"""
가공유류,가공치즈,감자 및 전분류,견과류,곡류 및 그 제품,곡류, 서류 제품,과·채가공품류,과실류,과일·채소류음료,과일류,과자,과자류,구이류,국 및 탕류,규격외일반가공식품,기타,기타음료,김치류,나물/숙채류,난류,다류,당류,당류 및 그 제품,두류 및 그 제품,두류, 견과 및 종실류,두부류 또는 묵류,두유류,드레싱류,땅콩 또는 견과류 가공품류,만두류,면 및 만두류,면류,발효유류,발효음료류,밥류,버섯류,버터류,복합조미식품,볶음류,분식,분유류,빵 또는 떡류,빵 및 과자류,생채/무침류,성장기용 조제식,소스류,소시지류,수조어육류,시리얼류,식물성크림,식용유지류,식육 또는 알가공품,식육 추출가공품,아이스크림,아이스크림류,아이스크림믹스류,어육가공품,어패류,엿류,영·유아용 곡류조제식,올리고당류,우유 및 유제품,우유가공품,우유류,유가공품,유당분해우유,유제품류 및 빙과류,유지류,유크림류,육류 및 그 제품,음료,음료 및 주류,음료 및 차류,인삼·홍삼음료,자연치즈,장기보존식품,장아찌/절임류,잼류,저지방우유류,전/ 적 및 부침류,젓갈류,제빵,조리가공식품류,조림류,조미김,조미료류,조제유류,죽 및 스프류,즉석섭취·편의식품류,찌개 및 전골류,찜류,채소, 해조류,채소류,체중조절용 조제식품,추출가공식품류,카레,커피,코코아가공품류 또는 초콜릿류,탄산음료류,튀김류,특수의료용도등식품,팝콘용옥수수가공품,피자,해조류,햄버거'
"""

"\n가공유류,가공치즈,감자 및 전분류,견과류,곡류 및 그 제품,곡류, 서류 제품,과·채가공품류,과실류,과일·채소류음료,과일류,과자,과자류,구이류,국 및 탕류,규격외일반가공식품,기타,기타음료,김치류,나물/숙채류,난류,다류,당류,당류 및 그 제품,두류 및 그 제품,두류, 견과 및 종실류,두부류 또는 묵류,두유류,드레싱류,땅콩 또는 견과류 가공품류,만두류,면 및 만두류,면류,발효유류,발효음료류,밥류,버섯류,버터류,복합조미식품,볶음류,분식,분유류,빵 또는 떡류,빵 및 과자류,생채/무침류,성장기용 조제식,소스류,소시지류,수조어육류,시리얼류,식물성크림,식용유지류,식육 또는 알가공품,식육 추출가공품,아이스크림,아이스크림류,아이스크림믹스류,어육가공품,어패류,엿류,영·유아용 곡류조제식,올리고당류,우유 및 유제품,우유가공품,우유류,유가공품,유당분해우유,유제품류 및 빙과류,유지류,유크림류,육류 및 그 제품,음료,음료 및 주류,음료 및 차류,인삼·홍삼음료,자연치즈,장기보존식품,장아찌/절임류,잼류,저지방우유류,전/ 적 및 부침류,젓갈류,제빵,조리가공식품류,조림류,조미김,조미료류,조제유류,죽 및 스프류,즉석섭취·편의식품류,찌개 및 전골류,찜류,채소, 해조류,채소류,체중조절용 조제식품,추출가공식품류,카레,커피,코코아가공품류 또는 초콜릿류,탄산음료류,튀김류,특수의료용도등식품,팝콘용옥수수가공품,피자,해조류,햄버거'\n"

In [ ]:
"""감자,과일,채소,나물/숙채류,난류,다류,당류,당류 및 그 제품,두류 및 그 제품,두류, 견과 및 종실류,두부류 
또는 묵류,두유류,드레싱류,땅콩 또는 견과류 가공품류,만두류,면 및 만두류,면류,발효유류,발효음료류,밥류,버섯류,
버터류,복합조미식품,볶음류,분식,분유류,빵 또는 떡류,빵 및 과자류,생채/무침류,성장기용 조제식,소스류,소시지류,
수조어육류,시리얼류,식물성크림,식용유지류,식육 또는 알가공품,식육 추출가공품,아이스크림,아이스크림류,아이스크림믹스류,어육가공품,어패류,엿류,영·유아용 곡류조제식,올리고당류,우유 및 유제품,우유가공품,우유류,유가공품,유당분해우유,유제품류 및 빙과류,유지류,유크림류,육류 및 그 제품,음료,음료 및 주류,음료 및 차류,인삼·홍삼음료,자연치즈,장기보존식품,장아찌/절임류,잼류,저지방우유류,전/ 적 및 부침류,젓갈류,제빵,조리가공식품류,조림류,조미김,조미료류,조제유류,죽 및 스프류,즉석섭취·편의식품류,찌개 및 전골류,찜류,채소, 해조류,채소류,체중조절용 조제식품,추출가공식품류,카레,커피,코코아가공품류 또는 초콜릿류,탄산음료류,튀김류,특수의료용도등식품,팝콘용옥수수가공품,피자,해조류,햄버거"""

In [10]:
# 작업결과 필터링 함수
import re
def temp_filter(food_token, food_nutr):
    food_nutr_names = list(food_nutr['식품이름'])
    food_nutr_temp  = ["".join(re.findall('[가-힣]+', str(_))) for _ in food_nutr_names]    
    food_nutr_token_index = [no  for no, _ in enumerate(food_nutr_temp)
                                 if _.find(food_token) != -1 ]
    # return len(food_nutr_token_index)
    if len(food_nutr_token_index) == 0: 
        return food_token
    else: pass # return len(food_nutr_token_index)

temp_filter("무", food_nutr)

In [11]:
%%time
temp_filter_result = [temp_filter(_, food_nutr)  for _ in food_items 
                                  if temp_filter(_, food_nutr)]
len(temp_filter_result)
# " ".join(food_nut_filter)

CPU times: user 9.24 s, sys: 12 ms, total: 9.26 s
Wall time: 9.26 s


262

In [12]:
",".join(temp_filter_result)

' ,가스오부시,가스오브시,가쓰오브시,갈비산적,감자고로캐,감자고로케,강정소스,거피들깨,건고추,건무말랭이,건시래기,건파래,계란지단,고추지,고춧잎무침,곱슬,곱슬이,기피들깨,김가루,민찌,참기름,김가루/,김가루/돈민찌,김가루/민찌/참기름,김가루/우동원액,김가루/참기름,김치/맛김가루,깐들깨,깐들깨분,깐쪽파,깨바라,깻바라,깻잎지,꽃빵,냉동야채,너비아니,닭갈비용,닭도리,닭뼈,닭정육,당근,오이,당근,피클,당근/대파,당근/레몬,당근/양파,대파,대파,당근,대파/맛김가루,대파채,도리닭,도리육,돈갈비,돈등뼈,돈목뼈,돈민지,돈민찌,돈민찌/김가루,돈민찌/김가루/참기름,돈불,돈불고기,돈사태,돈장조림,돈전지,돈제육,돈짜장,돈찌개,돈채,돈카레,돈후지,동부묵,동치미/사골,동치미맛,사골맛,동치미역수,동치미육수,두반장,들깨분,등뼈,딸기원액,땅콩분말,땅콩분태,떡국떡,라이스페이퍼,로메인,막국수면,맛김가루,맛김가루/민찌/참기름,맛김치,맛살,스모크,망고원액,매실분말,메란,명엽채,무 ,무/김가루,묵은지닭찜,미니찐빵,민물새우,민찌,민찌/김가루,민찌/김가루/참기름,민찌/참기름,밀떡,밀떡볶이,바지락살,반달단무지,방울새송이,베이키드빈,복숭아원액,볼어묵,봉어묵,부대찌개햄,부대콩,부추 ,부추,맛살,부추/당근,부추/미나리,부추산적,분홍소시지,블랙올리브,비빔야채,사각어묵,산고추,산적구이,살구원액,새송이/팽이,새송이곤약,새송이장조림,석류원액,스톡,닭뼈,실파,아삭이,알감자,알떡함박,알마늘,알마늘지,알어묵,알찬소시지,야채 ,야채재고,양배추  ,양배추/깻잎,양배추/양파/당근,양송이캔,양파 ,양파,당근,양파/당근,양파/피망,양파/피클,염장다시마,오렌지 ,오렌지원,오렌지원액,오복지,오이 ,오이,무초절,오징어/홍합,오징어까스,옥수수캔,온두부,와사비분,우국거리,우동고명,우동원액,우반골,우반골,잡뼈,우엉채,우잡뼈,원액,유린기소스,유부슬,유브슬,유자청,적날치알,적채,적채,목이,전장김,절단꽃게,절단오징어,제육김치,제육김치볶음,제육볶음,종합어묵,주키니,죽순/표고,죽순편,준순/표고,쥐어채,진미채,쪽파,쪽파김치,쫑상추,쭈꾸미,참기름/김가루,참

## **3 식재료 정보1**
미처리 단어들 사전 만들기
### **01 레시피 단어 목록**
식별 가능한 최소단위 DataBase

In [13]:
# 레시피 재료가 원재료와 다른경우 처리가 필요
food_recipe_token = sorted(set(food_menuset['식재료명']))
",".join(food_recipe_token)[::10]

' 오오,자,,건건파,,지적고잎곱이근김,김루가루김치만분깻,꽈,냉,무닭용정이/당파/데리묵돈지김루불,돈개후묵치미동두가딸가분조,라트늘로수가맛스액드추음,무순면,,민민,루바달울배이,묵콩추,산분,채,과적추팽송선액,대,스스게라래실몬안박,알야로장 배배이양양망언근다지원지,절,합어캔와르리,우기엉유부,멸,,어제음,,,채쪽,빵김,,찹챔지채청어초권커,,리보면콩,림위,토토,,슬인액애인프스팽섯,,고,,표실바클스할,그모씨리해박기,합,후크캔'

### **02 가락시장 450개 단어 목록**
식별 가능한 최소단위 DataBase

In [14]:
result = []
import re
import pandas as pd
food_token = pd.read_excel('data/food_Garak.xls')
# food_token = list(sorted(set(food_token['품목명'])))
# for _ in food_token:
#     result += re.findall(r"\w+", _)
# ",".join(list(sorted(set(result))))

food_token = """가리비,가시오이,가오리,가자미,가지,갈치,감,감귤,감숭어,감자,감홍,갑오징어,개량,
,거봉,멸치,겨자잎,고구마,고등어,고수,곰취나물,국산,굴,그린,극조생,근대,기타,김,쪽파,깻잎,꽁치,꽃게,
,버섯,고추,낙지,냉태,넙치,오이,노래미,농어,호박,무,다시마,단감,호박,달래,당근,대구,대추,대파,도루묵,
,돔,돗나물,딸기,레몬,로메인,멍게,메론,명태,바나나,무,무순,무화과,오징어,미나리,민어,바나나,바실,바지락,
,배,배추,오이,조기,병어,복숭아,배추,부사,부추,북어,브로콜리,블루베리,양배추,사과,삼치,상추,새우,조개,
,대추,표고,석류,세러리,수박,시금치,파,배추,쑥갓,아귀,아로니아,아보카도,아욱,배추,애호박,약시,양광,
,양배추,양상추,양송이,양파,열무,영양부추,오렌지,오만둥이,오이,오징어,우럭,마늘,근대,상추,전복,전어,
,조기,쪽파,상추,쭈꾸미,옥수수,참나물,참다래,숭어,참외,조기,청경채,상추,고추,피망,취나물,취청,치커리,
,케일,콜라비,토마토,파세리,파인애플,파프리카,포도,조개,마늘,호박,호박밤,홍어,피망,홍합,꽃게,장어,
,황도,히로사끼"""

food_token = sorted(set(food_token.split(",")))
",".join(food_token)[1:]

',가리비,가시오이,가오리,가자미,가지,갈치,감,감귤,감숭어,감자,감홍,갑오징어,개량,거봉,겨자잎,고구마,고등어,고수,고추,곰취나물,국산,굴,그린,극조생,근대,기타,김,깻잎,꽁치,꽃게,낙지,냉태,넙치,노래미,농어,다시마,단감,달래,당근,대구,대추,대파,도루묵,돔,돗나물,딸기,레몬,로메인,마늘,멍게,메론,멸치,명태,무,무순,무화과,미나리,민어,바나나,바실,바지락,배,배추,버섯,병어,복숭아,부사,부추,북어,브로콜리,블루베리,사과,삼치,상추,새우,석류,세러리,수박,숭어,시금치,쑥갓,아귀,아로니아,아보카도,아욱,애호박,약시,양광,양배추,양상추,양송이,양파,열무,영양부추,오렌지,오만둥이,오이,오징어,옥수수,우럭,장어,전복,전어,조개,조기,쪽파,쭈꾸미,참나물,참다래,참외,청경채,취나물,취청,치커리,케일,콜라비,토마토,파,파세리,파인애플,파프리카,포도,표고,피망,호박,호박밤,홍어,홍합,황도,히로사끼'

In [ ]:
"""가리비,가시오이,가오리,가자미,가지,갈치,감,감귤,감숭어,감자,감홍,갑오징어,개량,거봉,겨자잎,고구마,
,고등어,고수,고추,곰취나물,국산,굴,그린,극조생,근대,기타,김,깻잎,꽁치,꽃게,낙지,냉태,넙치,노래미,농어,
,다시마,단감,달래,당근,대구,돼지,대추,대파,도루묵,돔,돗나물,딸기,레몬,로메인,마늘,멍게,메론,멸치,명태,
,무,무순,무화과,미나리,민어,바나나,바실,바지락,배,배추,버섯,병어,복숭아,부사,부추,북어,브로콜리,블루베리,
,사과,삼치,상추,새우,석류,세러리,수박,숭어,시금치,쑥갓,아귀,아로니아,아보카도,아욱,애호박,약시,양광,
양배추,양상추,양송이,양파,열무,영양부추,오렌지,오만둥이,오이,오징어,옥수수,우럭,장어,전복,전어,조개
,조기,쪽파,쭈꾸미,참나물,참다래,참외,청경채,취나물,취청,치커리,케일,콜라비,토마토,파,파세리,파인애플
,파프리카,포도,표고,피망,호박,호박밤,홍어,홍합,황도,히로사끼"""

### **02 영양소 정보 연결**
우선 최소 연결범위로 database 완성하기

In [15]:
import pandas as pd
food_nutr = pd.read_excel("data/food_nutrient.xls")
food_nutr_names_org = ["".join(re.findall("[가-힣]+", str(_))) for _ in food_nutr['식품이름'].to_list()]
food_nutr_names = sorted(set(food_nutr_names_org))[1:]
",".join(food_nutr_names)[::450]

'가쿠치기,식것츄레기골땅크애쵸일그,김깻,드남레청,티콜운살,앤르,니넛강기장리유래따키만스퍼팝하든리가크로이송림일두국육모심기청,이초미복맛피초전합렛스위이리밥운쌍샐브고케파,통영만탕나목밥튀와생서설,곱또우,,개레있콘스,얼혜크바,아카찌림차강청강,양국티렛화강골를오사언어요가고장먹운,프캔낵위도행업안자국도원도벨과숭라올,림아쌀익스원늘것슈타,스슘치땅코이우믹,라,코푸경생코트블타라,,홍튀다드아,즌친시틀날피토피스직즈불오,마짱퍼비프스드바양트스찜삼호이햄아공이르도,덮후'

In [16]:
# 필터링 결과를 출력
def food_nutr_filter(token_filter, food_nutr_names):
    # 입력값이 없을 때
    if token_filter.strip() == "":
        return None
    token_filter_result = [ _ for _ in food_nutr_names   
                              if (_.find(token_filter) != -1) and (_.find("생것") != -1)]
    # 생것으로 추출 결과값이 없을 때
    if len(token_filter_result) == 0:
        token_filter_result = [ _ for _ in food_nutr_names  if _.find(token_filter) != -1]
    # 그래도 아무 결과값 없을 때
    if len(token_filter_result) == 0:
        return None
    print(token_filter_result)
    token_filter_index = food_nutr_names_org.index(token_filter_result[0])
    return food_nutr.iloc[token_filter_index, :]
    # token_filter_index = [food_nutr_names_org.index(_)  for _ in token_filter_result]
    # print(food_nutr.iloc[token_filter_index, :]['식품이름'].to_list())
    
food_nutr_filter("와사비", food_nutr_names)

['고추냉이가루와사비가루', '고추냉이와사비페이스트', '고추냉이페이스트와사비페이스트', '그린피스와사비스낵', '노리와사비땅콩과자', '노리텐와사비김함유', '녹차와사비완두콩', '땅콩과와사비맛감씨쌀과자', '사쿠사쿠와사비코', '새우와와사비센베이', '슈퍼후레쉬와사비카키노타네', '오부쿠로와사비맛마메그린피스', '오부쿠로와사비맛마메잠두', '오부쿠로츠마미이치방와사비그린피스', '오부쿠로츠마미이치방와사비잠두', '와사비그린마메', '와사비그린피스', '와사비마메', '와사비마메와사비그린마메', '와사비마메와사비빈스', '와사비맛그린피스스낵', '와사비맛차야와사비그린마메', '와사비맛차야와사비빈스', '와사비빈스', '와사비완두', '와사비완두콩', '와사비자야와사비그린마메', '와사비자야와사비빈스', '와사비카키피', '와사비푸미', '와사비피스타치오', '쯔마미와사비맛그린마메', '쯔마미와사비맛빈스', '쯔마미이찌방와사비그린마메', '쯔마미이찌방와사비맛그린마메', '쯔마미이찌방와사비맛빈스', '쯔마미이찌방와사비빈스', '쯔마미이치방와사비그린피', '쯔마미이치방와사비빈', '카키노다네와사비', '캬슈가이와사비그린마메']


번호                                  546
식품코드                 100106001800100001
식품군                                 채소류
식품이름                      고추냉이가루(와사비가루)
1회제공량(g)                            100
열량(kcal)(1회제공량당)                     81
탄수화물(g)(1회제공량당)                    23.7
단백질(g)(1회제공량당)                      2.9
지방(g)(1회제공량당)                       0.5
당류(g)(1회제공량당)                       NaN
나트륨(mg)(1회제공량당)                       1
콜레스테롤(mg)(1회제공량당)                   NaN
포화지방산(g)(1회제공량당)                    NaN
트랜스지방(g)(1회제공량당)                    NaN
가공업체명                               NaN
구축년도                               2017
자료원                     농촌진흥청  식품성분표 DB
Name: 545, dtype: object

### **03 레시피와 영양소 정보 연결**
우선 최소 연결범위로 database 완성하기

In [17]:
# 레시피 재료가 원재료와 다른경우 처리가 필요
food_recipe_token = sorted(set(food_menuset['식재료명']))
",".join(food_recipe_token)[::10]

' 오오,자,,건건파,,지적고잎곱이근김,김루가루김치만분깻,꽈,냉,무닭용정이/당파/데리묵돈지김루불,돈개후묵치미동두가딸가분조,라트늘로수가맛스액드추음,무순면,,민민,루바달울배이,묵콩추,산분,채,과적추팽송선액,대,스스게라래실몬안박,알야로장 배배이양양망언근다지원지,절,합어캔와르리,우기엉유부,멸,,어제음,,,채쪽,빵김,,찹챔지채청어초권커,,리보면콩,림위,토토,,슬인액애인프스팽섯,,고,,표실바클스할,그모씨리해박기,합,후크캔'

In [14]:
# 영양소 필터에서 연관목록 추출
food_nutr_filter("돼지살코기", food_nutr_names)

['돼지살코기생것']


번호                                 1590
식품코드                 100209001101300001
식품군                           육류 및 그 제품
식품이름                           돼지살코기,생것
1회제공량(g)                            100
열량(kcal)(1회제공량당)                    178
탄수화물(g)(1회제공량당)                       0
단백질(g)(1회제공량당)                    19.78
지방(g)(1회제공량당)                     11.25
당류(g)(1회제공량당)                         0
나트륨(mg)(1회제공량당)                      49
콜레스테롤(mg)(1회제공량당)                 62.82
포화지방산(g)(1회제공량당)                   3.62
트랜스지방(g)(1회제공량당)                    NaN
가공업체명                               NaN
구축년도                               2017
자료원                     농촌진흥청  식품성분표 DB
Name: 1589, dtype: object

In [18]:
token = {
    1:["난류","가금류","달걀"],
    2:"우유", 
    3:"메밀",
    4:"땅콩", 
    5:"대두", 
    6:"밀", 
    7:"고등어",
    8:"게",
    9:"새우",
    10:"돼지고기",
    11:"복숭아", 
    12:"토마토", 
    13:["아황산류"],
    14:"호두",
    15:"닭고기",
    16:"쇠고기", 
    17:"오징어", 
    18:"조개류(굴,전복,홍합 포함)",
}

In [21]:
# 작업결과 필터링 함수
import re
def food_nutrition_filter(food_token, food_nutr):
#     food_token = str(food_token).strip() # 공백 줄이기
    food_nutr_names = list(food_nutr['식품이름'])
    food_nutr_temp  = ["".join(re.findall('[가-힣]+', str(_))) for _ in food_nutr_names]    
    # food_token = "햄" : 구분용 Token 을 기준으로 영양정보 추출하기
    food_nutr_token_index = [no  for no, _ in enumerate(food_nutr_temp)
                             if _.find(food_token) != -1]
    #print(len(food_nutr_token_index))
    if food_nutr_token_index:
        # 결과 중 "생것" 포함시 필터링
        if re.findall(r"생것", "".join(list(food_nutr.iloc[food_nutr_token_index,:]['식품이름']))):
            food_nutr_token_index = [no  for no, _ in enumerate(food_nutr_temp)
                                     if _.find(food_token) != -1     if _.find('생것') != -1]
            if food_nutr_token_index:
                # 필터링 결과 중 Not Null 값이 많은 내용을 우선 추출
                index_notnull = [sum(food_nutr.iloc[_,:].notnull())  for _ in food_nutr_token_index]
                index_notnull_max = food_nutr_token_index[index_notnull.index(max(index_notnull))]
                if index_notnull_max:
                    return list(food_nutr.iloc[index_notnull_max, :])
                else:
                    return list(food_nutr.iloc[food_nutr_token_index[0], :])
        # 생것 필터링 결과가 없을 때
        else:
            return list(food_nutr.iloc[food_nutr_token_index[0], :])
    return food_token.strip()

food_nutrition_filter("무", food_nutr)

252


[640,
 100106006100100001,
 '채소류',
 '총각무무청,생것',
 100.0,
 21.0,
 '4.59',
 '1.97',
 '0.34',
 0.61,
 '68',
 0.0,
 0.07,
 nan,
 nan,
 2017,
 '농촌진흥청  식품성분표 DB']

In [22]:
%%time
food_nut_filter = [food_nutrition_filter(_, food_nutr)  for _ in food_items]
temp = pd.DataFrame(food_nut_filter)
temp.to_csv("test.csv")

0
0
0
10
0
18
16
0
96
0
0
2
0
0
0
0
2
8
0
0
10
28
24
3
5
1
11
0
126
2
11
4
0
2
0
5
0
0
7
1
124
35
3
0
434
2
0
0
0
0
0
0
11
2
171
0
4
0
0
0
0
0
24
0
8
10
0
1
1
58
1
0
57
0
1
5
13
12
46
3
135
9
0
0
0
0
18
0
0
0
0
0
13
0
0
0
0
6
9
0
0
12
0
16
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
19
0
0
0
0
0
17
0
81
7
8
1
0
0
0
98
1
2
0
0
7
0
33
1
3
0
3
71
0
10
16
2
8
0
0
0
0
30
0
48
0
0
5
0
1
14
35
8
0
3
252
0
0
9
1
4
0
16
10
0
1
4
9
0
0
0
0
0
7
0
0
9
0
0
6
0
75
31
8
69
0
0
0
2
0
0
0
9
0
0
0
0
0
15
1
0
9
0
0
11
0
9
1
109
6
0
0
0
9
5
0
0
0
14
8
16
0
15
120
13
4
4
19
13
2
11
8
3
1
0
67
3
1
12
4
68
1
1
0
3
103
0
3
14
0
0
0
0
1
0
0
3
112
0
1
0
10
14
0
0
0
1
3
15
0
29
0
0
0
0
0
23
75
9
11
6
28
0
1
85
0
0
0
3
0
47
0
0
1
2
95
0
0
3
66
0
0
50
0
2
113
2
0
0
3
0
0
0
24
2
10
0
0
0
0
16
0
0
1
0
22
10
0
0
0
0
0
0
10
0
0
0
2
0
0
0
0
0
2
1
0
2
0
0
0
9
0
0
3
15
0
0
28
2
76
0
1
0
0
0
0
0
1
0
0
2
0
3
0
0
0
0
0
3
3
192
1
0
0
38
0
0
0
0
32
4
6
3
0
0
20
0
5
0
17
0
0
1
12
4
61
0
0
2
0
2
0
36
0
0
0
0
0
0
0
4
4
7
0
6
0
5
0
22
0


In [23]:
undefined = [_  for _ in temp[0]  if type(_) == str]
len(undefined)

262

## **3 식재료 정보2**
식별되지 않은 262개 단어 필터링

In [ ]:
",//".join(undefined)

In [20]:
food_nutrition_filter(_, food_nutr)

NameError: name 'food_nutrition_filter' is not defined

In [ ]:
undefined_test = [food_nutrition_filter(_, food_nutr)  for _ in undefined]
undefined_test

In [19]:
food_nutrition_filter("짜장", food_nutr)

NameError: name 'food_nutrition_filter' is not defined

In [ ]:
undefined

In [ ]:
temp[0][0]

In [ ]:
# 갯수가 여럿 추출시 "생것" 포함을 우선추출
if len(food_nutr_token_index) > 1:
    if _.find('생것') != -1
food_nutr.loc[food_nutr_token_index,['식품군', '식품이름']]

In [ ]:
food_token = "찹쌀"
food_nutr_token_index = [no  for no, _ in enumerate(food_nutr_temp)
                         if _.find(food_token) != -1]
# 갯수가 여럿 추출시 "생것" 포함을 우선추출
if len(food_nutr_token_index) > 1:
    if _.find('생것') != -1
food_nutr.loc[food_nutr_token_index,['식품군', '식품이름']]

In [ ]:
print(food_nutr.iloc[[4451], :])

In [ ]:
import re
# for _ in food_items:
#     re.
    

In [ ]:
import re
food_nutr_names

In [ ]:
len(set(food_nutr['식품이름']))

# **레시피 데이터**
## **1 레시피 데이터 전처리**
데이터 프레임 자료 불러오기

In [ ]:
food_data['제육바싹볶음']

In [ ]:
foods = [_ for _ in food_data.keys()]

In [3]:
# import json
# json.loads('data/food_recipie.json')

In [ ]:
import pandas as pd
food_menu_org = pd.read_excel(file_list[1])
food_menu_org.shape, food_menu_org.head(2)

## **2 개별 레시피 Json 구분**
레시피 데이터 Json 재료와 전처리
1. 중분류 Tag 가 포함된 것도 있지만, 대부분은 재료만 포함
1. 우선 개별 list 객체로 세분화 하기
1. 추후에 중간묶음, 개별 재료와 단위명 구분하기
1. 이 작업은 블로그등 다양한 

In [ ]:
food_menu = food_menu_org.loc[:,["소유자","메뉴명","재료정보"]]
food_menu["재료정보"] = [str(_).splitlines() for _ in food_menu["재료정보"]]
food_menu.head(3)

In [ ]:
food_menu['재료정보'][2]

## **3 Detail 구분하기**
레시피 데이터 Json 재료와 전처리
```
tag_item = re.findall(r"\(.*?\)", temp)[0]
```

In [ ]:
recipe_temp = food_menu['재료정보'][5]
token_temp  = recipe_temp[0]
token_temp  = "참깨 5g(조금),  소금 조금"
token_temp.split(',')

In [ ]:
import re
re.findall(r"\w+", token_temp)

In [ ]:
re.findall(r"\(.*?\)", token_temp)

In [ ]:
re.findall(r"\d+g", token_temp)

In [ ]:
import re
re.findall(r"(조금|약간)", token_temp)

In [ ]:
re.findall(r"\d+g", token_temp)
token_temp.split('(')[0]

## **4 메뉴명 공통분모 적용해보기**
기존에 작업한 기준의 성능 평가하기

In [1]:
with open("data/nouns_tokens.txt" , "r") as f:
    nouns_token = f.read()
nouns_token = nouns_token.split(",")
len(nouns_token)

686

In [ ]:
menu_org = list(food_menu_org.메뉴명)

In [ ]:
from muyong.nlp import food_nouns
menu_valid = [["_".join(food_nouns(_, nouns_token)), _]  for _ in menu_org]

In [ ]:
import pandas as pd
menu_names = pd.DataFrame(menu_valid)

In [ ]:
menu_names.to_csv("menu_tokens.csv")

In [ ]:
# Plus Token
스시, 영양밥, 관자요리, 청국장, 고소, 담백, 파르페 ,자장면, 돌나물, 전복, 화이타, 아보카도
카나페, 크런치롤, 구기자, 월남쌈, 오코노미야키, 크러스트, 크로켓, 우둔살, 스틱, 수란, 삼합
카프레제, 인삼, 초계, 백김치 ,토란, 배숙, 누룽, 크로켓, 차우더, 부각, 롤, 전병, 황태포, 면, 월남쌈, 리조또, 아란치니,
육회, 대하, 삼계, 약식, 카나페, 쇠고기, 해산물, 차돌박이, 대하, 양갱, 게살, 딤섬,무스, 수정과, 아스파라거스 ,크로켓,
비지, 나가사키,  (361 계속작업)

In [ ]:
# More Unique (나누기)
들깨소스, 비트, 가자미조림

# **레시피 데이터 작업용 함수**
## **1 레시피 데이터 전처리**
단일한 기준으로 자료 정리하기
1. **메뉴명** 과 **식재료** 기준 정하기
1. **어떤 작업을 해야할 지 애매해서** 방황하는 부분이 많았음!!!
1. **만개의 레시피** 등과 같은 자료를 통해 **공통분모** 추출하기

In [ ]:
from muyong.nlp import food_nouns
# food_nouns()

In [ ]:
from glob import glob
file_list = glob("data/food_*.xls")
file_list

In [ ]:
import pandas as pd
food_ingre_org = pd.read_excel(file_list[4])
food_ingre_org.shape, food_ingre_org.head(2)

In [ ]:
len(set(food_ingre_org.식품이름))

In [ ]:
list(food_ingre_org.식품이름)

In [ ]:
food_ingre_org.columns

In [ ]:
test = {
    0:"python",
    1:"tensorflow"
}

In [ ]:
test.get(4, 0)

In [ ]:
def double(x):
    return x * 2

In [ ]:
list(map(double, [1,2,3,4]))

In [ ]:
import re
def menu_recipe(menu1):
    #menu1 = food_recipe[0]
    menu_detail = menu1.splitlines()[1]
    menu_detail = [_.strip() for _ in menu_detail.split(',')]
    temp     = menu_detail[0].split(" ")[1]
    tag_item = re.findall(r"\(.*?\)", temp)[0]
    menu_detail[0].split(" ")[0], temp.replace(tag_item, ""), tag_item
    result = []
    for _ in menu_detail:
        temp     = _.split(" ")[1]
        tag_item = re.findall(r"\(.*?\)", temp)[0]
        result.append([menu1.splitlines()[0], 
                       _.split(" ")[0], 
                       temp.replace(tag_item, ""), 
                       tag_item])
    return result